# Searching the Catalog
__________________

The EarthOne Catalog API is a single interface through which you can discover existing raster datasets, search and retrieve their associated images, and manage your own datasets.

This guide is meant to serve as an introduction to searching and retrieving raster data. For a more in depth overview of all Catalog classes and their capabilities please visit the [API Reference](https://docs.earthone.earthdaily.com/earthdaily/earthone/catalog/readme.html) and [Catalog Guide](https://docs.earthone.earthdaily.com/guides/catalog.html) sections in our Documentation page.

In [ ]:
import earthdaily.earthone as eo
from earthdaily.earthone.catalog import Image, Product, properties as p

## Catalog Products
The foundational class within Catalog is a [`Product`](https://docs.earthone.earthdaily.com/earthdaily/earthone/catalog/docs/product.html#earthdaily.earthone.catalog.Product). You can think of a product as a collection of imagery from the same data source, such as [Sentinel-2](https://sentinel.esa.int/web/sentinel/missions/sentinel-2), but more specifically with the same processesing level such as  [Sentinel-2 Level 2A](https://sentinels.copernicus.eu/web/sentinel/user-guides/sentinel-2-msi/product-types/level-2a).

Below we will start with the product ID for [Sentinel-2 L2A](https://app.earthone.earthdaily.com/explorer/datasets/esa:sentinel-2:l2a:v1). You can find product IDs available to you through our graphical user interface [Explorer](https://earthone.earthdaily.com/explorer/) or through a programmatic [`Search`](https://docs.earthone.earthdaily.com/earthdaily/earthone/catalog/docs/search.html#earthdaily.earthone.catalog.Search).

In [ ]:
s2_product = Product.get("esa:sentinel-2:l2a:c1:v1")
s2_product

A product contains a number of [`Image`s](https://docs.earthone.earthdaily.com/earthdaily/earthone/catalog/docs/image.html#earthdaily.earthone.catalog.Image), which themselves contain the same number of [`Band`s](https://docs.earthone.earthdaily.com/earthdaily/earthone/catalog/docs/band.html#earthdaily.earthone.catalog.Band). 

Let's see how many images and bands there are in this product through accessing the respective [`Product.images()`](https://docs.earthone.earthdaily.com/earthdaily/earthone/catalog/docs/product.html#earthdaily.earthone.catalog.Product.images) and [`Product.bands()`](https://docs.earthone.earthdaily.com/earthdaily/earthone/catalog/docs/product.html#earthdaily.earthone.catalog.Product.bands) methods. Note that the type of each is their respective Search object:

In [ ]:
s2_images = s2_product.images()
s2_bands = s2_product.bands()

print(f"Number of Images: {s2_images.count()}")
print(f"Type: {type(s2_images)}")
print(f"Number of Bands: {s2_bands.count()}")
print(f"Type: {type(s2_bands)}")

## GeoContexts
Once a product is identified, searching over spatiotemporal extents is simple. The foundational [`GeoContext`](https://docs.earthone.earthdaily.com/earthdaily/earthone/geo/readme.html#earthdaily.earthone.geo.GeoContext) class is an [`AOI`](https://docs.earthone.earthdaily.com/earthdaily/earthone/geo/readme.html#earthdaily.earthone.geo.AOI). An AOI specifies certain spatial parameters which define the returned raster dataset such as resolution, coordinate reference system, and a geometry cutline.

Here we will create an AOI from a geojson geometry over the Washington, DC area:

In [ ]:
geometry = {
    "type": "Polygon",
    "coordinates": [
        [
            [-77.06237256372073, 38.83807622810744],
            [-76.96982073518272, 38.83807622810744],
            [-76.96982073518272, 38.921715779114066],
            [-77.06237256372073, 38.921715779114066],
            [-77.06237256372073, 38.83807622810744],
        ]
    ],
}

aoi = eo.geo.AOI(geometry, resolution=30.0, crs="EPSG:3857")
aoi

Note, we could also pass a shapely geometry here if we preferred.

## Finding Images
Now that we have both a product identified and an AOI over which we want to search imagery, we can chain a series of [`filter`s](https://docs.earthone.earthdaily.com/earthdaily/earthone/catalog/docs/search.html#earthdaily.earthone.catalog.Search.filter) to return an [`ImageCollection`](https://docs.earthone.earthdaily.com/earthdaily/earthone/catalog/docs/image.html#earthdaily.earthone.catalog.ImageCollection). 

Let's find all the Sentinel-2 images captured in the month of June, 2023 with less than 10% cloud cover over Washington, DC:

In [ ]:
search = (
    s2_images.intersects(aoi)
    .filter("2023-06-01" < p.acquired < "2023-07-01")
    .filter(p.cloud_fraction < 0.1)
)

image_collection = search.collect()
image_collection

There are other attributes useful to filter by, documented in the API reference for [`Image`](https://docs.earthone.earthdaily.com/earthdaily/earthone/catalog/docs/image.html#earthdaily.earthone.catalog.Image).

## Rastering Data - Mosaics
Our [`ImageCollection`](https://docs.earthone.earthdaily.com/earthdaily/earthone/catalog/docs/image.html#earthdaily.earthone.catalog.ImageCollection) now has all of the methods we need to retrieve our pixel data. 

First we'll explore [`ImageCollection.mosaic()`](https://docs.earthone.earthdaily.com/earthdaily/earthone/catalog/docs/image.html#earthdaily.earthone.catalog.ImageCollection.mosaic), which will load all specified bands into a single 3D ndarray:

In [ ]:
rgb = image_collection.mosaic(bands=["red", "green", "blue"])
eo.utils.display(rgb, size=5)

__*Note:*__ Here we used each image's default __scaling__ and __data_type__ for retrieving our pixel data. [Refer to our Documentation for more advanced usage of these parameters.](https://docs.earthone.earthdaily.com/earthdaily/earthone/catalog/docs/image.html#earthdaily.earthone.catalog.ImageCollection.scaling_parameters)

The default shape of the resulting ndarray is __(nbands, ny, nx)__:

In [ ]:
rgb.shape

Alternatively, call [`ImageCollection.download_mosaic()`](https://docs.earthone.earthdaily.com/earthdaily/earthone/catalog/docs/image.html#earthdaily.earthone.catalog.ImageCollection.download_mosaic) to download this as a geotiff:

In [ ]:
image_collection.download_mosaic(bands=["red", "green", "blue"], dest="data/rgb.tif")

## Rastering Data - Stacks
If we instead wanted to retrieve each image as a 4D ndarray we use [`ImageCollection.stack()`](https://docs.earthone.earthdaily.com/earthdaily/earthone/catalog/docs/image.html#earthdaily.earthone.catalog.ImageCollection.stack) to return a ndarray of shape __(nimages, nbands, ny, nx)__:

In [ ]:
rgb_stack = image_collection.stack(bands=["red", "green", "blue"])
rgb_stack.shape

## ImageCollection Properties
We can iterate over the properties of our stack as well:

In [ ]:
dates = list(image_collection.each.acquired.strftime("%Y-%m-%d %H-%m-%s"))
dates

Let's take a look at each image to explore the data in more detail:

In [ ]:
ids = list(image_collection.each.id)
titles = [f"{ids[i]}\n{dates[i]}" for i in range(len(ids))]

eo.utils.display(*rgb_stack, title=titles, size=5)

Note that each image may not completely overlap our input geometry, as these are individual Sentinel-2 scenes.